In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime

# Step 1: Fetch Data
start_date = "2010-01-01"
end_date = "2024-12-31"
risk_free_ticker = "^IRX"  # Use the 13-week Treasury yield as a proxy for risk-free rate
market_index_ticker = "^GSPC"  # S&P 500 index
equity_tickers = ["AAPL", "MSFT", "GOOGL", "TSLA", "NVDA"]  # Replace with desired stock tickers

# Fetch data
risk_free_data = yf.download(risk_free_ticker, start=start_date, end=end_date)["Close"]
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)["Close"]
stock_data = yf.download(equity_tickers, start=start_date, end=end_date)["Close"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed


In [7]:
# import matplotlib.pyplot as plt

# # Assuming stock_data['NVDA'] contains the data for NVIDIA's stock
# plt.figure(figsize=(10, 6))  # Optional: Adjusts the figure size
# plt.plot(stock_data['NVDA'], label='NVIDIA Stock Prices')
# plt.title('NVIDIA Stock Price Over Time')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.legend()
# plt.grid(True)
# plt.show()

import plotly.graph_objects as go

# Assuming stock_data['NVDA'] contains the data for NVIDIA's stock
fig = go.Figure()

# Add a line plot for NVIDIA stock prices
fig.add_trace(go.Scatter(
    x=stock_data.index,  # Assuming the index contains dates
    y=stock_data['NVDA'],
    mode='lines',
    name='NVIDIA Stock Prices'
))

# Add a line plot for NVIDIA stock prices
fig.add_trace(go.Scatter(
    x=stock_data.index,  # Assuming the index contains dates
    y=stock_data['TSLA'],
    mode='lines',
    name='TSLA Stock Prices'
))

# Customize the layout
fig.update_layout(
    title='NVIDIA Stock Price Over Time',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_white',
    showlegend=True
)

# Show the plot
fig.show()

In [8]:
# Step 2: Prepare Data
# Calculate daily returns
market_returns = market_data.pct_change().dropna()
stock_returns = stock_data.pct_change().dropna()

# Calculate SMB and HML
# For simplicity, create synthetic SMB and HML factors
smb = stock_returns.mean(axis=1)  # Proxy for SMB
top_50 = stock_returns.quantile(0.5, axis=1)
hml = top_50 - stock_returns.mean(axis=1)  # Proxy for HML

In [9]:
hml

Date
2010-06-30   -0.002162
2010-07-01    0.003814
2010-07-02    0.022579
2010-07-06    0.027358
2010-07-07    0.008000
                ...   
2024-12-23    0.001553
2024-12-24   -0.011819
2024-12-26    0.001779
2024-12-27    0.005780
2024-12-30   -0.000460
Length: 3650, dtype: float64

In [10]:
smb

Date
2010-06-30   -0.015950
2010-07-01   -0.015940
2010-07-02   -0.029269
2010-07-06   -0.028458
2010-07-07    0.024403
                ...   
2024-12-23    0.015270
2024-12-24    0.021193
2024-12-26   -0.004380
2024-12-27   -0.023082
2024-12-30   -0.012779
Length: 3650, dtype: float64

In [11]:
# convert to pandas dataframe
market_returns = pd.DataFrame(market_returns).dropna()
stock_returns = pd.DataFrame(stock_returns).dropna()

# Calculate SMB
# SMB is typically calculated as the return of small-cap stocks minus large-cap stocks.
# For simplicity, we'll proxy SMB as the average return of stocks.
smb = stock_returns.mean(axis=1)  # Average return of all stocks, as a simple SMB proxy

# Calculate HML
# HML is the return of high book-to-market (value) stocks minus low book-to-market (growth) stocks.
# For simplicity, we'll use the top 50th percentile return as a proxy for high and average return for the mean.
top_50 = stock_returns.quantile(0.5, axis=1)  # Median return as a simple high proxy
hml = top_50 - stock_returns.mean(axis=1)  # Proxy for HML

stock_returns['smb'] = smb.values
stock_returns['top_50'] = top_50.values
stock_returns['hml'] = hml.values

In [12]:
# Convert risk-free rate from percentage to daily rate
risk_free_rate = risk_free_data / 100 / 252
risk_free_rate = risk_free_rate.reindex(market_returns.index, method="ffill")

# convert to pandas dataframe
risk_free_rate = pd.DataFrame(risk_free_rate).dropna()

In [24]:
# Merge All data together
merged_df = pd.merge(stock_returns, 
                    pd.merge(market_returns, risk_free_rate,  
                            left_index=True, right_index=True),
                    left_index=True, right_index=True)

for col in merged_df.columns:
    merged_df[f"{col}_cumu_return"] = (1 + merged_df[col]).cumprod()

In [25]:
merged_df.head()

Ticker,AAPL,GOOGL,MSFT,NVDA,TSLA,smb,top_50,hml,^GSPC,^IRX,AAPL_cumu_return,GOOGL_cumu_return,MSFT_cumu_return,NVDA_cumu_return,TSLA_cumu_return,smb_cumu_return,top_50_cumu_return,hml_cumu_return,^GSPC_cumu_return,^IRX_cumu_return
Date,,,,,,,,,,,,,,,,,,,,
2010-06-30,-0.018113,-0.020495,-0.012870,-0.025763,-0.002511,-0.015950,-0.018113,-0.002162,-0.010113,0.000007,0.981887,0.979505,0.987130,0.974237,0.997489,0.984050,0.981887,0.997838,0.989887,1.000007
2010-07-01,-0.012126,-0.012271,0.006519,0.016650,-0.078473,-0.015940,-0.012126,0.003814,-0.003240,0.000007,0.969981,0.967485,0.993565,0.990458,0.919213,0.968364,0.969981,1.001644,0.986679,1.000013
2010-07-02,-0.006197,-0.006690,0.004750,-0.012524,-0.125683,-0.029269,-0.006690,0.022579,-0.004662,0.000006,0.963970,0.961013,0.998285,0.978053,0.803683,0.940021,0.963492,1.024260,0.982079,1.000020
2010-07-06,0.006844,-0.001099,0.023636,-0.010732,-0.160937,-0.028458,-0.001099,0.027358,0.005359,0.000006,0.970567,0.959957,1.021880,0.967557,0.674341,0.913270,0.962433,1.052282,0.987342,1.000026
2010-07-07,0.040381,0.032403,0.020151,0.048324,-0.019243,0.024403,0.032403,0.008000,0.031331,0.000006,1.009760,0.991062,1.042471,1.014313,0.661364,0.935557,0.993619,1.060700,1.018276,1.000032


In [33]:
selected = 'smb'
comparable = 'hml'
# Assuming merged_df['NVDA'] contains the data for NVIDIA's stock
fig = go.Figure()

# Add a line plot for NVIDIA stock prices
fig.add_trace(go.Scatter(
    x=merged_df.index,  # Assuming the index contains dates
    y=merged_df[f'{selected}_cumu_return'],
    mode='lines',
    name=f'{selected} Cumulative Return'
))

if comparable != '':
    # Add a line plot for NVIDIA stock prices
    fig.add_trace(go.Scatter(
        x=merged_df.index,  # Assuming the index contains dates
        y=merged_df[f'{comparable}_cumu_return'],
        mode='lines',
        name=f'{comparable} Cumulative Return'
    ))

    # Customize the layout
    fig.update_layout(
        title=f'{selected} and {comparable} Daily Returm',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_white',
        showlegend=True
    )
else:
    # Customize the layout
    fig.update_layout(
        title=f'{selected} Daily Returm',
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_white',
        showlegend=True
    )

# Show the plot
fig.show()

In [34]:
# Calculate market excess return
merged_df['market_excess_return'] = merged_df['^GSPC'] - merged_df['^IRX']
merged_df['dt'] = merged_df.index

In [35]:
merged_df.dtypes

Ticker
AAPL                           float64
GOOGL                          float64
MSFT                           float64
NVDA                           float64
TSLA                           float64
smb                            float64
top_50                         float64
hml                            float64
^GSPC                          float64
^IRX                           float64
AAPL_cumu_return               float64
GOOGL_cumu_return              float64
MSFT_cumu_return               float64
NVDA_cumu_return               float64
TSLA_cumu_return               float64
smb_cumu_return                float64
top_50_cumu_return             float64
hml_cumu_return                float64
^GSPC_cumu_return              float64
^IRX_cumu_return               float64
market_excess_return           float64
dt                      datetime64[ns]
dtype: object

In [36]:
merged_df['dt'].max()

Timestamp('2024-12-30 00:00:00')

In [37]:
# Train / Test Split

TRAIN_df = merged_df[merged_df['dt'] <= '2023-12-31']
TEST_df = merged_df[merged_df['dt'] > '2023-12-31']

print(TRAIN_df['dt'].max(), TRAIN_df['dt'].min())
print(TEST_df['dt'].max(), TEST_df['dt'].min())

2023-12-29 00:00:00 2010-06-30 00:00:00
2024-12-30 00:00:00 2024-01-02 00:00:00


In [38]:
# CAPM
factors = ['market_excess_return']
results = {}

for stock in equity_tickers:
    y = TRAIN_df[stock] - TRAIN_df['^IRX']
    x = TRAIN_df[factors]
    model = sm.OLS(y, x).fit()
    results[stock] = model

test_rf = TEST_df['^IRX'].mean()
test_market_excess_return = TEST_df['market_excess_return'].mean()

# Output results
for stock, model in results.items():
    print(f"Regression Results for {stock}:")
    # print(model.summary())
    print(model.params)
    CAPM_expected = test_rf + model.params[0] * test_market_excess_return
    stock_actual_mean = TEST_df[stock].mean()
    print(f"for {stock}, actual average return is {stock_actual_mean}, and CAPM expected return is {CAPM_expected}")
    print("\n")

Regression Results for AAPL:
market_excess_return    1.113185
dtype: float64
for AAPL, actual average return is 0.0011970940394644613, and CAPM expected return is 0.0009617735651914056


Regression Results for MSFT:
market_excess_return    1.132876
dtype: float64
for MSFT, actual average return is 0.0005874096681219588, and CAPM expected return is 0.0009753063733719035


Regression Results for GOOGL:
market_excess_return    1.098459
dtype: float64
for GOOGL, actual average return is 0.001422001993258815, and CAPM expected return is 0.0009516528928139745


Regression Results for TSLA:
market_excess_return    1.411795
dtype: float64
for TSLA, actual average return is 0.00284925071743566, and CAPM expected return is 0.0011669986952688107


Regression Results for NVDA:
market_excess_return    1.61002
dtype: float64
for NVDA, actual average return is 0.004617465794986664, and CAPM expected return is 0.00130323287522112




/tmp/ipykernel_4878/374020102.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [39]:
# Three-factor
factors = ['market_excess_return', 'smb', 'hml']
results = {}

for stock in equity_tickers:
    y = TRAIN_df[stock] - TRAIN_df['^IRX']
    x = TRAIN_df[factors]
    model = sm.OLS(y, x).fit()
    results[stock] = model

test_rf = TEST_df['^IRX'].mean()
test_market_excess_return = TEST_df['market_excess_return'].mean()
test_smb = TEST_df['smb'].mean()
test_hml = TEST_df['hml'].mean()


# Output results
for stock, model in results.items():
    print(f"Regression Results for {stock}:")
    # print(model.summary())
    print(model.params)
    Three_model_expected = test_rf + model.params[0] * test_market_excess_return + + model.params[1] * test_smb + + model.params[2] * test_hml
    print(f"for {stock}, actual average return is {stock_actual_mean}, and CAPM expected return is {Three_model_expected}")
    print("\n")

Regression Results for AAPL:
market_excess_return    0.201009
smb                     0.752337
hml                     0.563948
dtype: float64
for AAPL, actual average return is 0.004617465794986664, and CAPM expected return is 0.0013397046169251674


Regression Results for MSFT:
market_excess_return    0.291719
smb                     0.705029
hml                     0.696576
dtype: float64
for MSFT, actual average return is 0.004617465794986664, and CAPM expected return is 0.0011596898285093757


Regression Results for GOOGL:
market_excess_return    0.189799
smb                     0.756117
hml                     0.666439
dtype: float64
for GOOGL, actual average return is 0.004617465794986664, and CAPM expected return is 0.001230821513779905


Regression Results for TSLA:
market_excess_return   -0.650587
smb                     1.501212
hml                    -1.855167
dtype: float64
for TSLA, actual average return is 0.004617465794986664, and CAPM expected return is 0.0049316179996

/tmp/ipykernel_4878/3211501656.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [17]:
alpha = model.params[0]
print(alpha)

1.0882598082971544


/tmp/ipykernel_2386/2133321700.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]


In [ ]:
# Step 3: Run Regression for Each Stock and Test

results = {}
factors = ['smb', 'hml', 'market_excess_return']
for stock in equity_tickers:
    y = merged_df[stock] - merged_df['^IRX']
    x = sm.add_constant(merged_df[factors])
    model = sm.OLS(y, x).fit()
    results[stock] = model

# Output results
for stock, model in results.items():
    print(f"Regression Results for {stock}:")
    print(model.summary())
    print("\n")

In [16]:
all_data = pd.DataFrame(stock_returns).dropna()

In [ ]:
all_data.columns

In [7]:
# Calculate market excess return
market_excess_return = market_returns - risk_free_rate

In [ ]:
market_excess_return

In [ ]:
risk_free_rate = risk_free_data / 100 / 252


In [ ]:
yf.download(['TSLA'], start=start_date, end=end_date)

In [ ]:
IRX